### INNOVATHON

# juntar temperaturas de todos los años

In [18]:
import pandas as pd
import glob
import os

# Carpeta donde están los archivos Excel
carpeta = "innovation moderate\hdd-anual"  

# Buscar todos los archivos Excel (.xlsx o .xls)
archivos = glob.glob(os.path.join(carpeta, "*.xlsx")) + glob.glob(os.path.join(carpeta, "*.xls"))

# Leer y concatenar todos los archivos
df = pd.concat([pd.read_excel(f) for f in archivos], ignore_index=True)

# Eliminar duplicados (opcional)
df = df.drop_duplicates()
#sobreescribir los valores la columna Unnamed: 0 a Date en los casos que no tiene contenido 
df['Date'] = df.apply(lambda row: row['Unnamed: 0'] if pd.isnull(row['Date']) else row['Date'], axis=1)
df = df.drop(columns=['Unnamed: 0'])
# Guardar el resultado
df.to_csv(os.path.join("total-degree-days.xlsx"), index=False)




<>:6: SyntaxWarning: invalid escape sequence '\h'
<>:6: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_5712\4033492573.py:6: SyntaxWarning: invalid escape sequence '\h'
  carpeta = "innovation moderate\hdd-anual"


# juntar instalaciones con dos calderas en 1 

In [11]:
import pandas as pd

# --- 1. Leer los dos archivos Excel ---
df1 = pd.read_excel("innovation moderate\produccion-energetica\m218820-1.xlsx")
df2 = pd.read_excel("innovation moderate\produccion-energetica\m218820-2.xlsx")

# --- 2. Asegurar que las columnas tienen el mismo nombre ---
df1.columns = ["Timestamp", "EnergiaCaldera1(MW-hr)"]
df2.columns = ["Timestamp", "EnergiaCaldera2(MW-hr)"]

# --- 6. Sumar las dos columnas de energía ---
df1["EnergiaCaldera1(MW-hr)"] = df1["EnergiaCaldera1(MW-hr)"].astype(str).str.replace(",", ".").astype(float)
df2["EnergiaCaldera2(MW-hr)"] = df2["EnergiaCaldera2(MW-hr)"].astype(str).str.replace(",", ".").astype(float)

df_sum = pd.merge(df1, df2, on="Timestamp", how="outer", suffixes=("_1", "_2"))

df_sum["EnergiaCaldera(MW-hr)"] = df1["EnergiaCaldera1(MW-hr)"] + df2["EnergiaCaldera2(MW-hr)"]

# --- 7. Dejar solo las dos columnas finales ---
df_final = df_sum[["Timestamp", "EnergiaCaldera(MW-hr)"]]

# --- 8. Ordenar por Timestamp (opcional) ---
df_final = df_final.sort_values("Timestamp")

# --- 5. (Opcional) Guardar resultado a un nuevo Excel ---
df_final.to_excel("m218820.xlsx", index=False)

print(df_sum.head())


                  Timestamp  EnergiaCaldera1(MW-hr)  EnergiaCaldera2(MW-hr)  \
0  1 abr 2024 00:00:00 CEST                    98.4                    61.3   
1  1 abr 2025 00:00:00 CEST                   188.8                   193.7   
2  1 ago 2024 00:00:00 CEST                   119.9                    82.3   
3  1 ago 2025 00:00:00 CEST                   209.0                   213.9   
4   1 dic 2024 00:00:00 CET                   144.3                   103.5   

   EnergiaCaldera(MW-hr)  
0                  125.8  
1                  126.8  
2                  127.8  
3                  128.8  
4                  129.8  


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:5: SyntaxWarning: invalid escape sequence '\p'
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7076\2023588663.py:4: SyntaxWarning: invalid escape sequence '\p'
  df1 = pd.read_excel("innovation moderate\produccion-energetica\m218820-1.xlsx")
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7076\2023588663.py:5: SyntaxWarning: invalid escape sequence '\p'
  df2 = pd.read_excel("innovation moderate\produccion-energetica\m218820-2.xlsx")


# aemet


In [12]:
import requests
import xml.etree.ElementTree as ET

url = "https://www.aemet.es/xml/municipios/localidad_37274.xml"
response = requests.get(url)
root = ET.fromstring(response.content)


In [13]:
datos = []
for dia in root.findall(".//dia"):
    fecha = dia.attrib["fecha"]
    tmax = float(dia.find("temperatura/maxima").text)
    tmin = float(dia.find("temperatura/minima").text)
    datos.append({"fecha": fecha, "tmax": tmax, "tmin": tmin})


In [14]:
base_temp = 18
for d in datos:
    tmedia = (d["tmax"] + d["tmin"]) / 2
    d["HDD"] = max(0, base_temp - tmedia)


In [15]:
import pandas as pd

df = pd.DataFrame(datos)
print(df)
df.to_excel("degree_days_aemet.xlsx", index=False)

        fecha  tmax  tmin  HDD
0  2025-10-24  21.0  11.0  2.0
1  2025-10-25  17.0  12.0  3.5
2  2025-10-26  18.0   6.0  6.0
3  2025-10-27  19.0   3.0  7.0
4  2025-10-28  21.0   7.0  4.0
5  2025-10-29  16.0  12.0  4.0
6  2025-10-30  15.0   8.0  6.5
